<a href="https://colab.research.google.com/github/kessingtonosazee/GCP_Project_1/blob/master/Robotics_Asssesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 2 Python File**

In [ ]:
#!/usr/bin/env python3

# imports rospy module and twist message type
import rospy
from geometry_msgs.msg import Twist
import math

class MoveTurtleBot():

    # initialize class attributes
    def __init__(self):
        self.turtlebot_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1) # creates a publisher for cmd_vel topic, specify twist message, sets queue size
        self.cmd = Twist()   # initializes a twist object
        self.ctrl_c = False  # initializes a flag for shutdown
        self.rate = rospy.Rate(1) # initializes the rate of publising
        rospy.on_shutdown(self.shutdownhook) # sets the on_shutdown method to shutdownhook function when it is shutting down.

    def publish_once_in_cmd_vel(self):
        """
        This is because publishing in topics sometimes fails the first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important as it ensures the velocity command is published atleast once.
        """
        while not self.ctrl_c:
            connections = self.turtlebot_vel_publisher.get_num_connections()
            if connections > 0:
                self.turtlebot_vel_publisher.publish(self.cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()

    def shutdownhook(self):
        # works better than the rospy.is_shutdown()
        self.stop_turtlebot()
        self.ctrl_c = True

    def stop_turtlebot(self):
        rospy.loginfo("shutdown time! Stop the robot")
        self.cmd.linear.x = 0.0
        self.cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel()

    def MoveTurtleBot(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        self.cmd.linear.x = linear_speed
        self.cmd.angular.z = angular_speed

        i = 0
        rospy.loginfo("Moving TurtleBot!")
        while not self.ctrl_c and i <= moving_time:
            self.publish_once_in_cmd_vel()
            i = i+1
            self.rate.sleep()

        self.stop_turtlebot()

    def MoveRectangle(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        for _ in range(4):
            self.cmd.linear.x = linear_speed
            self.cmd.angular.z = 0.0
            i = 0

            rospy.loginfo("Moving TurtleBot in a straight line")
            while not self.ctrl_c and i <= moving_time:
                self.publish_once_in_cmd_vel()
                i = i+1
                self.rate.sleep()

            # Stop briefly
            self.stop_turtlebot()
            rospy.sleep(1.0)

            # turns 90 degrees to the right
            desired_angle_degrees = 90.0
            time_duration_seconds = 5.0

            # Convert degrees to radians
            desired_angle_radians = desired_angle_degrees*(math.pi/180.0) # (180 degrees = pi radians)

            # Calculate angular speed
            angular_speed = desired_angle_radians / time_duration_seconds

            # Set linear and angular speed values
            self.cmd.linear.x = 0.0
            self.cmd.angular.z = angular_speed

            # log and publish to topic
            rospy.loginfo("TurtleBot turns 90 degrees to the right!")
            self.publish_once_in_cmd_vel()
            rospy.sleep(5.0)


            # Stop briefly
            self.stop_turtlebot()
            rospy.sleep(1.0)

    def MoveCircle(self, duration=10, radius=1.0):

        # Calculate linear velocity to achieve a complete circle
        linear_speed = 2 * math.pi * radius / duration

        # Calculate angular velocity based on linear velocity and radius
        angular_speed = linear_speed / radius

        # Set values for linear and angular speed
        self.cmd.linear.x = linear_speed
        self.cmd.angular.z = angular_speed

        # log info
        rospy.loginfo(f"Moving TurtleBot in a circle for {duration} seconds.")
        start_time = rospy.get_time()

        while rospy.get_time() - start_time <= duration:
            self.turtlebot_vel_publisher.publish(self.cmd)
            self.rate.sleep()

        self.stop_turtlebot()
        rospy.sleep(1.0)


if __name__ == '__main__':
    rospy.init_node('move_turtlebot_test', anonymous=True)
    moveturtlebot_object = MoveTurtleBot()

    try:
        moveturtlebot_object.MoveRectangle(10)
        moveturtlebot_object.MoveCircle(10, 1.0)

    except rospy.ROSInterruptException:
        pass

**Question 2 Bash File**

In [ ]:
ARG1=$1

if [ "$ARG1" == "circle" ]; then
   echo "circling";
   rosrun rss_linux_pkg move_turtlebot_circle.py

elif [ "$ARG1" == 'forward_backward' ]; then
     echo "back and forth";
     rosrun rss_linux_pkg move_turtlebot_forward_backward.py

elif [ "$ARG1" == 'rectangle_circle' ]; then
     echo "rectangle and circle";
     rosrun rss_linux_pkg move_turtlebot_rectangle_circle.py
else
echo "Please enter one of the following;
circle
forward_backward
rectangle_circle"

fi

**Question 1**

In [ ]:
#!/usr/bin/env python3

# initiate variable
sum_result = 0

# define the range: 0 : 99
number_set = range(0,100)

# Calculate the sum from 0 to 99
for number_index in number_set:
    sum_result += number_index

# Print Result
print('Sum of numbers from 0 to 99:', sum_result)

**Question 1 Continued**
Detail what commands you used and for what purpose ?
1. touch
2. vim
3. chmod
4. ./

##Question 3

In [ ]:
#!/usr/bin/env python3

#import modules and packages
import rospy
from geometry_msgs.msg import Twist

#initialize the ROS node
rospy.init_node('osazeeogbeide_publisher_line')

#create a publisher for twist message
pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

#rate at which message is published
rate = rospy.Rate(1)

#create a twist message with linear velocity along the x-axis
move = Twist()
move.linear.x = 0.5
move.angular.z = 0.0  # No angular velocity for moving straight

while not rospy.is_shutdown():
    pub.publish(move)
    rate.sleep()
####################################################################################################################

<launch>

<node name="oz_pub_line" pkg="rss_pubsub_pkg" type="osazeeogbeide_publisher_line.py" output="screen"/>

</launch>
###################################################################################################################
# line subscriber
#!/usr/bin/env python3

import rospy
from std_msgs.msg import Int32
from geometry_msgs.msg import Twist

def callback(msg):
    print("Linear velocity is")
    print(msg.linear)
    print("Angular velocity is")
    print(msg.angular)

#rospy initializes the subscriber node
rospy.init_node("osazeeogbeide_subscriber_line")

# declares that the node subscribes to the '/cmd_vel'topic  of type: Twist
sub = rospy.Subscriber("/cmd_vel", Twist, callback )

# keeps node from exiting until the node has been shut down
rospy.spin()
####################################################################################################################

# publisher
#!/usr/bin/env python3

#libraries to set robot velocity
import rospy
from geometry_msgs.msg import Twist

#Enables communication between the node: pengwang_publisher and ROS master
rospy.init_node("osazeeogbeide_publisher")

#Defines the publisher to publish on topic:/cmd_vel with message type: Twist
pub = rospy.Publisher("/cmd_vel", Twist, queue_size=1)

#Defines the frequency of running the loop
rate = rospy.Rate(1)

#Specifies the message to publish to '/cmd_vel' topic
move = Twist()
move.linear.x = 0.5
move.angular.z = 0.5

#Always check 'rospy.is_shutdown()'
while not rospy.is_shutdown():
    pub.publish(move)
    rate.sleep()
######################################################################################################################


##Question 4

In [ ]:
#!/usr/bin/env python3

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from std_msgs.msg import Int32
from tf.transformations import euler_from_quaternion
import math

class OsazeeOgbeidePubSub:
    def __init__(self):
        # Initialize ROS node
        rospy.init_node('osazeeogbeide_pubsub')

        # create Subscriber for Odometry
        self.odom_sub = rospy.Subscriber('/odom', Odometry, self.odom_callback)

        # create Publisher for Twist messages
        self.cmd_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

        # set frequency
        self.rate = rospy.Rate(1) # 1Hz

        # Create Twist message
        self.twist_msg = Twist()

        # Initialize variables
        self.start_position = None
        self.distance_threshold = 0.2 # Set the distance threshold to 0.2 meter
        self.is_stopped = False

    def odom_callback(self, odom_msg):

        # Extract position from Odometry message
        position = odom_msg.pose.pose.position
        x = position.x
        y = position.y

        # Calculate distance from start position
        if self.start_position is None:
            self.start_position = (x, y)
        distance = math.sqrt((x - self.start_position[0])**2 + (y - self.start_position[1])**2)

        # Check if distance threshold is reached
        if distance >= self.distance_threshold and not self.is_stopped:
            # Stop the robot
            self.twist_msg.linear.x = 0.0  # m/s
            self.twist_msg.angular.z = 0.0 # rads/s
            self.is_stopped = True

        elif distance < self.distance_threshold and self.is_stopped:
            # Reset velocity if distance is below threshold after stopping
            self.is_stopped = False

        # If not stopped, move the robot forward
        if not self.is_stopped:
            self.twist_msg.linear.x = 0.2  # 0.2 m/s
            self.twist_msg.angular.z = 0.0  # 0 rad/s

        # Publish Twist message
        self.cmd_vel_pub.publish(self.twist_msg)

        # Display distance. linear velocity, angular velocity covered
        rospy.loginfo("Distance covered: {:.2f} meters".format(distance))
        rospy.loginfo("Linear velocity: {:.2f} m/s".format(self.twist_msg.linear.x))
        rospy.loginfo("Angular velocity: {:.2f} rads/s".format(self.twist_msg.linear.z))

        # Sleep to maintain publishing rate
        self.rate.sleep()

    def run(self):
        # Keep the node running
        rospy.spin()

if __name__ == '__main__':
    osazeeogbeide_pubsub = OsazeeOgbeidePubSub()
    osazeeogbeide_pubsub.run()
#######################################################################################################################
<launch>

<node name="oz_pubsub_node" pkg="rss_pubsub_pkg" type="osazeeogbeide_pubsub.py" output="screen" />

</launch>


**Question 4 Method 2**

In [ ]:
#!/usr/bin/env python

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from math import sqrt

class OsazeeKessPubSub:
    def __init__(self):
      # Initialize ROS node
        rospy.init_node('osazee_pubsub_node', anonymous=True)

        # create Subscriber for Odometry
        self.odom_subscriber = rospy.Subscriber("/odom", Odometry, self.odom_callback)

        # create Publisher for Twist messages
        self.cmd_vel_publisher = rospy.Publisher("/cmd_vel", Twist, queue_size=10)

        # set frequency
        self.rate = rospy.Rate(1) # 1Hz

        # set maximum distance to run
        self.distance_to_run = 5.0  # Adjust this value to the distance you want the robot to run

        # set initial velocity
        self.linear_velocity = 0.2  # Adjust this value to set the linear velocity

        # set current distance
        self.current_distance = 0.0

        # set initial position and orientation
        self.initial_pose = None

    def odom_callback(self, odom):
        if self.initial_pose is None:

            self.initial_pose = odom.pose.pose.position

        # Extract position from Odometry message
        current_pose = odom.pose.pose.position

        # Calculate distance from initial position
        self.current_distance = sqrt((current_pose.x - self.initial_pose.x)**2 +
                                     (current_pose.y - self.initial_pose.y)**2)

        # Check if distance threshold is reached
        if self.current_distance >= self.distance_to_run:
            rospy.loginfo("Reached the desired distance. Stopping the robot.")
            self.stop_robot()
        else:
            self.move_robot()

    def move_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = self.linear_velocity
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)
        # Sleep to maintain publishing rate
        self.rate.sleep()

    def stop_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = 0.0
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)
        # Sleep to maintain publishing rate
        self.rate.sleep()

    def run(self):
      # Keep the node running
      rospy.spin()

if __name__ == '__main__':
    try:
        osazee_pubsub = OsazeeKessPubSub()
        rospy.spin()
    except rospy.ROSInterruptException:
        pass
###################################################################################################################

<launch>

    <node name="osazee_pubsub_node" pkg="rss_pubsub_pkg" type="osazee_pubsub.py" output="screen">
    </node>

</launch>


In [ ]:
#!/usr/bin/env python3

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from math import sqrt

class OsazeePubSub:
    def __init__(self):
        # Initialize ROS node
        rospy.init_node("osazee_pubsub", anonymous=True)

        # create Subscriber for Odometry
        self.odom_subscriber = rospy.Subscriber("/odom", Odometry, self.odom_callback)

        # create Publisher for Twist messages
        self.cmd_vel_publisher = rospy.Publisher("/cmd_vel", Twist, queue_size=10)

        # set maximum distance to run
        self.distance_to_run = 1.0  # Adjust this value to the distance you want the robot to run

        # set initial velocity
        self.linear_velocity = 0.4  # Adjust this value to set the linear velocity

        # set initial position and orientation
        self.initial_pose = None

    def odom_callback(self, odom):
        if self.initial_pose is None:
            self.initial_pose = odom.pose.pose.position

        # Extract position from Odometry message
        current_pose = odom.pose.pose.position

        # Calculate distance from initial position
        self.current_distance = sqrt((current_pose.x - self.initial_pose.x)**2 + (current_pose.y - self.initial_pose.y)**2)

        # Check if distance threshold is reached
        if self.current_distance >= self.distance_to_run:
            rospy.loginfo("Reached the desired distance. Stopping the robot.")
            self.stop_robot()
        else:
            self.move_robot()

    def move_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = self.linear_velocity
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)

    def stop_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = 0.0
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)

    def run(self):
        rospy.spin()

if __name__ == '__main__':
    osazee_pubsub = OsazeePubSub()
    osazee_pubsub.run()


**Question 4 Perfect**

In [ ]:
#!/usr/bin/env python3

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from math import sqrt

class OsazeePubSub:
    def __init__(self):
        # Initialize ROS node
        rospy.init_node("osazee_pubsub", anonymous=True)

        # create Subscriber for Odometry
        self.odom_subscriber = rospy.Subscriber("/odom", Odometry, self.odom_callback)

        # create Publisher for Twist messages
        self.cmd_vel_publisher = rospy.Publisher("/cmd_vel", Twist, queue_size=10)

        # set maximum distance to run
        self.distance_to_run = 1.0  # Adjust this value to the distance you want the robot to run

        # set initial velocity
        self.linear_velocity = 0.4  # Adjust this value to set the linear velocity

        # set initial position and orientation
        self.initial_pose = None
        self.stopped = False

    def odom_callback(self, odom):
        if self.initial_pose is None:
            self.initial_pose = odom.pose.pose.position

        # Extract position from Odometry message
        current_pose = odom.pose.pose.position

        # Calculate distance from initial position
        self.current_distance = sqrt((current_pose.x - self.initial_pose.x)**2 + (current_pose.y - self.initial_pose.y)**2)

        # Check if distance threshold is reached
        if self.current_distance >= self.distance_to_run:
            if not self.stopped:
                self.stopped = True
                self.stop_robot()
                rospy.loginfo("Reached the desired distance. Stopping the robot.")
        else:
            self.stopped = False
            self.move_robot()

    def move_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = self.linear_velocity
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)

    def stop_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = 0.0
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)

    def run(self):
        rospy.spin()

if __name__ == '__main__':
    osazee_pubsub = OsazeePubSub()
    osazee_pubsub.run()


In [ ]:
# trial

#!/usr/bin/env python3

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from math import sqrt

class OsazeePubSub:
    def __init__(self):
        # Initialize ROS node
        rospy.init_node("osazee_pubsub", anonymous=True)

        # create Subscriber for Odometry
        self.odom_subscriber = rospy.Subscriber("/odom", Odometry, self.odom_callback)

        # create Publisher for Twist messages
        self.cmd_vel_publisher = rospy.Publisher("/cmd_vel", Twist, queue_size=10)

        # set maximum distance to run
        self.distance_to_run = 1.0  # Adjust this value to the distance you want the robot to run

        # set initial velocity
        self.linear_velocity = 0.4  # Adjust this value to set the linear velocity

        # set initial position and orientation
        self.initial_pose = None

    def odom_callback(self, odom):
        if self.initial_pose is None:
            self.initial_pose = odom.pose.pose.position

        # Extract position from Odometry message
        current_pose = odom.pose.pose.position

        # Calculate distance from initial position
        self.current_distance = sqrt((current_pose.x - self.initial_pose.x)**2 + (current_pose.y - self.initial_pose.y)**2)

        # Check if distance threshold is reached
        if self.current_distance >= self.distance_to_run:
            self.stop_robot()
            rospy.loginfo("Reached the desired distance. Stopping the robot.")
        else:
            self.move_robot()

    def move_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = self.linear_velocity
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)

    def stop_robot(self):
        twist_msg = Twist()
        twist_msg.linear.x = 0.0
        twist_msg.angular.z = 0.0
        self.cmd_vel_publisher.publish(twist_msg)

    def run(self):
        rospy.spin()

if __name__ == '__main__':
    osazee_pubsub = OsazeePubSub()
    osazee_pubsub.run()


##QUESTION 5

In [ ]:
<launch>

    <node name="oz_pub_node" pkg="rss2_msgsrv_pkg" type="msg_pub.py" output="screen">
    </node>

</launch>
########################################################################################################################

<launch>

    <node name="oz_sub_node" pkg="rss2_msgsrv_pkg" type="msg_sub.py" output="screen">
    </node>

</launch>


In [1]:
#!/usr/bin/env python3

import rospy
from rss2_msgsrv_pkg.msg import date_cmd_vel
from geometry_msgs.msg import Twist
from datetime import datetime

rospy.init_node("msg_pub")

now = datetime.now()
date_str = now.strftime("%m/%d/%y, %H:%M:%S")

oz_cmd_vel = date_cmd_vel()
oz_cmd_vel.oz_date = date_str
oz_cmd_vel.oz_cmd_vel.linear.x = 0.5
oz_cmd_vel.oz_cmd_vel.angular.z = 0.1

pub = rospy.Publisher("/oz_topic", date_cmd_vel , queue_size =1)

rate = rospy.Rate(1)

while not rospy.is_shutdown():
    pub.publish(oz_cmd_vel)
    rate.sleep()

ModuleNotFoundError: No module named 'rospy'

In [ ]:
#!/usr/bin/env python3

import rospy

from rss2_msgsrv_pkg.msg import date_cmd_vel

def callback(msg):
    rospy.loginfo(msg.oz_date)
    rospy.loginfo(msg.oz_cmd_vel)

rospy.init_node("msg_sub")
sub = rospy.Subscriber("/oz_topic", date_cmd_vel ,callback)

rospy.spin()

In [2]:
# add the dependency to xml


<build_depend>geometry_msgs</build_depend>
  <build_depend>nav_msgs</build_depend>
  <build_depend>rospy</build_depend>
  <build_depend>std_msgs</build_depend>
  <build_depend>rss2_msgsrv_pkg</build_depend>
  <build_export_depend>geometry_msgs</build_export_depend>
  <build_export_depend>nav_msgs</build_export_depend>
  <build_export_depend>rospy</build_export_depend>
  <build_export_depend>std_msgs</build_export_depend>
  <build_export_depend>rss2_msgsrv_pkg</build_export_depend>
  <exec_depend>geometry_msgs</exec_depend>
  <exec_depend>nav_msgs</exec_depend>
  <exec_depend>rospy</exec_depend>
  <exec_depend>std_msgs</exec_depend>
  <exec_depend>rss2_msgsrv_pkg</exec_depend>

SyntaxError: invalid syntax (<ipython-input-2-194a34fb0ac4>, line 2)

In [ ]:
find_package(catkin REQUIRED COMPONENTS
  geometry_msgs
  nav_msgs
  rospy
  std_msgs
  rss2_msgsrv_pkg
)

##CMList

In [ ]:
cmake_minimum_required(VERSION 3.0.2)
project(rss2_msgsrv_pkg)

## Compile as C++11, supported in ROS Kinetic and newer
# add_compile_options(-std=c++11)

## Find catkin macros and libraries
## if COMPONENTS list like find_package(catkin REQUIRED COMPONENTS xyz)
## is used, also find other catkin packages
find_package(catkin REQUIRED COMPONENTS
  geometry_msgs
  nav_msgs
  rospy
  std_msgs
  rss2_msgsrv_pkg
)

## System dependencies are found with CMake's conventions
# find_package(Boost REQUIRED COMPONENTS system)


## Uncomment this if the package has a setup.py. This macro ensures
## modules and global scripts declared therein get installed
## See http://ros.org/doc/api/catkin/html/user_guide/setup_dot_py.html
# catkin_python_setup()

################################################
## Declare ROS messages, services and actions ##
################################################

## To declare and build messages, services or actions from within this
## package, follow these steps:
## * Let MSG_DEP_SET be the set of packages whose message types you use in
##   your messages/services/actions (e.g. std_msgs, actionlib_msgs, ...).
## * In the file package.xml:
##   * add a build_depend tag for "message_generation"
##   * add a build_depend and a exec_depend tag for each package in MSG_DEP_SET
##   * If MSG_DEP_SET isn't empty the following dependency has been pulled in
##     but can be declared for certainty nonetheless:
##     * add a exec_depend tag for "message_runtime"
## * In this file (CMakeLists.txt):
##   * add "message_generation" and every package in MSG_DEP_SET to
##     find_package(catkin REQUIRED COMPONENTS ...)
##   * add "message_runtime" and every package in MSG_DEP_SET to
##     catkin_package(CATKIN_DEPENDS ...)
##   * uncomment the add_*_files sections below as needed
##     and list every .msg/.srv/.action file to be processed
##   * uncomment the generate_messages entry below
##   * add every package in MSG_DEP_SET to generate_messages(DEPENDENCIES ...)

## Generate messages in the 'msg' folder
  add_message_files(
    FILES
    date_cmd_vel.msg
#   Message1.msg
#   Message2.msg
 )

## Generate services in the 'srv' folder
# add_service_files(
#   FILES
#   Service1.srv
#   Service2.srv
# )

## Generate actions in the 'action' folder
# add_action_files(
#   FILES
#   Action1.action
#   Action2.action
# )

## Generate added messages and services with any dependencies listed here
  generate_messages(
    DEPENDENCIES
    geometry_msgs
    nav_msgs
    std_msgs
 )

################################################
## Declare ROS dynamic reconfigure parameters ##
################################################

## To declare and build dynamic reconfigure parameters within this
## package, follow these steps:
## * In the file package.xml:
##   * add a build_depend and a exec_depend tag for "dynamic_reconfigure"
## * In this file (CMakeLists.txt):
##   * add "dynamic_reconfigure" to
##     find_package(catkin REQUIRED COMPONENTS ...)
##   * uncomment the "generate_dynamic_reconfigure_options" section below
##     and list every .cfg file to be processed

## Generate dynamic reconfigure parameters in the 'cfg' folder
# generate_dynamic_reconfigure_options(
#   cfg/DynReconf1.cfg
#   cfg/DynReconf2.cfg
# )

###################################
## catkin specific configuration ##
###################################
## The catkin_package macro generates cmake config files for your package
## Declare things to be passed to dependent projects
## INCLUDE_DIRS: uncomment this if your package contains header files
## LIBRARIES: libraries you create in this project that dependent projects also need
## CATKIN_DEPENDS: catkin_packages dependent projects also need
## DEPENDS: system dependencies of this project that dependent projects also need
catkin_package(
#  INCLUDE_DIRS include
#  LIBRARIES rss2_msgsrv_pkg
   CATKIN_DEPENDS geometry_msgs nav_msgs rospy std_msgs message_runtime
#  DEPENDS system_lib
)

###########
## Build ##
###########

## Specify additional locations of header files
## Your package locations should be listed before other locations
include_directories(
# include
  ${catkin_INCLUDE_DIRS}
)

## Declare a C++ library
# add_library(${PROJECT_NAME}
#   src/${PROJECT_NAME}/rss2_msgsrv_pkg.cpp
# )

## Add cmake target dependencies of the library
## as an example, code may need to be generated before libraries
## either from message generation or dynamic reconfigure
# add_dependencies(${PROJECT_NAME} ${${PROJECT_NAME}_EXPORTED_TARGETS} ${catkin_EXPORTED_TARGETS})

## Declare a C++ executable
## With catkin_make all packages are built within a single CMake context
## The recommended prefix ensures that target names across packages don't collide
# add_executable(${PROJECT_NAME}_node src/rss2_msgsrv_pkg_node.cpp)

## Rename C++ executable without prefix
## The above recommended prefix causes long target names, the following renames the
## target back to the shorter version for ease of user use
## e.g. "rosrun someones_pkg node" instead of "rosrun someones_pkg someones_pkg_node"
# set_target_properties(${PROJECT_NAME}_node PROPERTIES OUTPUT_NAME node PREFIX "")

## Add cmake target dependencies of the executable
## same as for the library above
# add_dependencies(${PROJECT_NAME}_node ${${PROJECT_NAME}_EXPORTED_TARGETS} ${catkin_EXPORTED_TARGETS})

## Specify libraries to link a library or executable target against
# target_link_libraries(${PROJECT_NAME}_node
#   ${catkin_LIBRARIES}
# )

#############
## Install ##
#############

# all install targets should use catkin DESTINATION variables
# See http://ros.org/doc/api/catkin/html/adv_user_guide/variables.html

## Mark executable scripts (Python etc.) for installation
## in contrast to setup.py, you can choose the destination
# catkin_install_python(PROGRAMS
#   scripts/my_python_script
#   DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION}
# )

## Mark executables for installation
## See http://docs.ros.org/melodic/api/catkin/html/howto/format1/building_executables.html
# install(TARGETS ${PROJECT_NAME}_node
#   RUNTIME DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION}
# )

## Mark libraries for installation
## See http://docs.ros.org/melodic/api/catkin/html/howto/format1/building_libraries.html
# install(TARGETS ${PROJECT_NAME}
#   ARCHIVE DESTINATION ${CATKIN_PACKAGE_LIB_DESTINATION}
#   LIBRARY DESTINATION ${CATKIN_PACKAGE_LIB_DESTINATION}
#   RUNTIME DESTINATION ${CATKIN_GLOBAL_BIN_DESTINATION}
# )

## Mark cpp header files for installation
# install(DIRECTORY include/${PROJECT_NAME}/
#   DESTINATION ${CATKIN_PACKAGE_INCLUDE_DESTINATION}
#   FILES_MATCHING PATTERN "*.h"
#   PATTERN ".svn" EXCLUDE
# )

## Mark other files for installation (e.g. launch and bag files, etc.)
# install(FILES
#   # myfile1
#   # myfile2
#   DESTINATION ${CATKIN_PACKAGE_SHARE_DESTINATION}
# )

#############
## Testing ##
#############

## Add gtest based cpp test target and link libraries
# catkin_add_gtest(${PROJECT_NAME}-test test/test_rss2_msgsrv_pkg.cpp)
# if(TARGET ${PROJECT_NAME}-test)
#   target_link_libraries(${PROJECT_NAME}-test ${PROJECT_NAME})
# endif()

## Add folders to be run by python nosetests
# catkin_add_nosetests(test)

##XML

In [ ]:
<?xml version="1.0"?>
<package format="2">
  <name>rss2_msgsrv_pkg</name>
  <version>0.0.0</version>
  <description>The rss2_msgsrv_pkg package</description>

  <!-- One maintainer tag required, multiple allowed, one person per tag -->
  <!-- Example:  -->
  <!-- <maintainer email="jane.doe@example.com">Jane Doe</maintainer> -->
  <maintainer email="ad22557868@todo.todo">ad22557868</maintainer>


  <!-- One license tag required, multiple allowed, one license per tag -->
  <!-- Commonly used license strings: -->
  <!--   BSD, MIT, Boost Software License, GPLv2, GPLv3, LGPLv2.1, LGPLv3 -->
  <license>TODO</license>


  <!-- Url tags are optional, but multiple are allowed, one per tag -->
  <!-- Optional attribute type can be: website, bugtracker, or repository -->
  <!-- Example: -->
  <!-- <url type="website">http://wiki.ros.org/rss2_msgsrv_pkg</url> -->


  <!-- Author tags are optional, multiple are allowed, one per tag -->
  <!-- Authors do not have to be maintainers, but could be -->
  <!-- Example: -->
  <!-- <author email="jane.doe@example.com">Jane Doe</author> -->


  <!-- The *depend tags are used to specify dependencies -->
  <!-- Dependencies can be catkin packages or system dependencies -->
  <!-- Examples: -->
  <!-- Use depend as a shortcut for packages that are both build and exec dependencies -->
  <!--   <depend>roscpp</depend> -->
  <!--   Note that this is equivalent to the following: -->
  <!--   <build_depend>roscpp</build_depend> -->
  <!--   <exec_depend>roscpp</exec_depend> -->
  <!-- Use build_depend for packages you need at compile time: -->
  <!--   <build_depend>message_generation</build_depend> -->
  <!-- Use build_export_depend for packages you need in order to build against this package: -->
  <!--   <build_export_depend>message_generation</build_export_depend> -->
  <!-- Use buildtool_depend for build tool packages: -->
  <!--   <buildtool_depend>catkin</buildtool_depend> -->
  <!-- Use exec_depend for packages you need at runtime: -->
  <!--   <exec_depend>message_runtime</exec_depend> -->
  <!-- Use test_depend for packages you need only for testing: -->
  <!--   <test_depend>gtest</test_depend> -->
  <!-- Use doc_depend for packages you need only for building documentation: -->
  <!--   <doc_depend>doxygen</doc_depend> -->
  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>geometry_msgs</build_depend>
  <build_depend>nav_msgs</build_depend>
  <build_depend>rospy</build_depend>
  <build_depend>std_msgs</build_depend>
  <build_export_depend>geometry_msgs</build_export_depend>
  <build_export_depend>nav_msgs</build_export_depend>
  <build_export_depend>rospy</build_export_depend>
  <build_export_depend>std_msgs</build_export_depend>
  <exec_depend>geometry_msgs</exec_depend>
  <exec_depend>nav_msgs</exec_depend>
  <exec_depend>rospy</exec_depend>
  <exec_depend>std_msgs</exec_depend>
  <exec_depend>message_runtime</exec_depend>



  <!-- The export tag contains other, unspecified, tags -->
  <export>
    <!-- Other tools can request additional information be placed here -->

  </export>
</package>

In [ ]:
# client

#!/usr/bin/env python3

import rospy
from rss2_msgsrv_pkg.srv import srv_turtlebot_move, srv_turtlebot_moveRequest

# The service client node
rospy.init_node('turtlebot_move_client')

# Wait for the service '/turtlebot_move_service' to run
# You need to start the service first
rospy.wait_for_service('/turtlebot_move_service')

# Connect to the service '/turtlebot_move_service'
turtlebot_service_client = rospy.ServiceProxy('/turtlebot_move_service', srv_turtlebot_move)

# Create a request instance
turtlebot_request_instance = srv_turtlebot_moveRequest()
turtlebot_request_instance.duration = 20

# Send the request to the server through connection built
feedback = turtlebot_service_client(turtlebot_request_instance)

# Show results after the service being called
rospy.loginfo(str(feedback))
rospy.loginfo('End of service call')


In [ ]:
# server

#!/usr/bin/env python3

import rospy
from rss2_msgsrv_pkg.srv import srv_turtlebot_move, srv_turtlebot_moveResponse
from geometry_msgs.msg import Twist

def my_callback(request):
    rospy.loginfo('Turtlebot_move_service has been called')

    vel = Twist()
    vel.linear.x = 0.2
    vel.angular.z = 0.2
    total_time = 0

    while total_time <= request.duration:
        oz_pub.publish(vel)
        rospy.loginfo('time = %d', total_time)
        rate.sleep()
        total_time += 1
        vel.linear.x = 0.0
        vel.angular.z = 0.0
        oz_pub.publish(vel)
        rate.sleep()

    return srv_turtlebot_moveResponse(True)

if __name__ == '__main__':
    rospy.init_node('turtlebot_move_server')

    oz_service = rospy.Service('/turtlebot_move_service', srv_turtlebot_move, my_callback)
    oz_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
    vel = Twist()
    rate = rospy.Rate(1)

    rospy.loginfo('Service /turtlebot_move_service is ready!')

    rospy.spin()
